In [ ]:
!pip install -q transformers datasets peft accelerate evaluate rouge-score nltk sacrebleu torch optimum[onnxruntime]

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.8/425.8 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.5 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("BinKhoaLe1812/MedDialog-EN-100k")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/118 [00:00<?, ?B/s]

HealthCareMagic-100k.json:   0%|          | 0.00/144M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112165 [00:00<?, ? examples/s]

In [ ]:
if 'validation' not in dataset:
    split_dataset = dataset['train'].train_test_split(test_size=0.1, seed=42)
    dataset['train'] = split_dataset['train']
    dataset['validation'] = split_dataset['test']

In [ ]:
print(f"Training: {len(dataset['train']):,}")
print(f"Validation: {len(dataset['validation']):,}")

Training: 100,948
Validation: 11,217


### Load Tokenizer

In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 27.2 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
import torch
from transformers import AutoModelForCausalLM

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/biogpt",
    torch_dtype=torch.float16,
    device_map="auto",
)

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

### Setup LoRA

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
lora_config = LoraConfig(
    r=16,  # Good rank for quality
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # All 4 for quality
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

In [ ]:
model = get_peft_model(model, lora_config)
model.enable_input_require_grads()

In [ ]:
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())

In [ ]:
print(f"Trainable: {trainable:,} ({100*trainable/total:.2f}%)")

Trainable: 2,359,296 (0.68%)


Preprocess

In [ ]:
def preprocess_function(examples):
    texts = []
    for i in range(len(examples['instruction'])):
        instruction = examples['instruction'][i] or ""
        input_text = examples['input'][i] if 'input' in examples and examples['input'][i] else ""
        output_text = examples['output'][i] or ""

        prompt = f"{instruction} {input_text}".strip() if input_text else instruction.strip()
        text = f"{prompt}\n{output_text}"
        texts.append(text)

    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=512,  # Full length for quality
        padding="max_length",
        return_tensors="pt"
    )
    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized

In [ ]:
tokenized_train = dataset['train'].map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['train'].column_names,
    desc="Train",
    num_proc=2
)

Train (num_proc=2):   0%|          | 0/100948 [00:00<?, ? examples/s]

In [ ]:
tokenized_eval = dataset['validation'].map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['validation'].column_names,
    desc="Val",
    num_proc=2
)

Val (num_proc=2):   0%|          | 0/11217 [00:00<?, ? examples/s]

### Balanced Training Setup

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import numpy as np

In [ ]:
training_args = TrainingArguments(
    output_dir="./biogpt-lora-balanced",
    per_device_train_batch_size=8,  # Balanced
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,  # Effective batch = 16
    learning_rate=2e-4,  # Standard LR
    num_train_epochs=1,  # Full epochs for quality
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    warmup_steps=100,
    fp16=True,
    gradient_checkpointing=True,  # For memory
    dataloader_num_workers=2,
    dataloader_pin_memory=True,
    push_to_hub=False,
    report_to="none",
)

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
class PerplexityTrainer(Trainer):
    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
        metrics = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
        if f"{metric_key_prefix}_loss" in metrics:
            metrics[f"{metric_key_prefix}_perplexity"] = np.exp(metrics[f"{metric_key_prefix}_loss"])
        return metrics

In [ ]:
trainer = PerplexityTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


### Train

In [ ]:
train_result = trainer.train()

Step,Training Loss,Validation Loss
500,3.359100,3.246274
1000,3.232400,3.146441
1500,3.186400,3.092566
2000,3.143200,3.057678
2500,3.138900,3.031778
3000,3.107300,3.010683
3500,3.104600,2.998122
4000,3.080200,2.984522
4500,3.082100,2.975530
5000,3.060300,2.965659


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [ ]:
print(f"Loss: {train_result.training_loss:.4f}")

Loss: 3.1545


### Evaluate

In [ ]:
eval_results = trainer.evaluate()

In [ ]:
print(f"Loss: {eval_results['eval_loss']:.4f}")
print(f"Perplexity: {eval_results['eval_perplexity']:.4f}")

Loss: 2.9576
Perplexity: 19.2522


### Save Original + Merged

In [ ]:
# Save LoRA adapter (small)
trainer.save_model("./biogpt-lora-balanced")
tokenizer.save_pretrained("./biogpt-lora-balanced")

('./biogpt-lora-balanced/tokenizer_config.json',
 './biogpt-lora-balanced/special_tokens_map.json',
 './biogpt-lora-balanced/vocab.json',
 './biogpt-lora-balanced/merges.txt',
 './biogpt-lora-balanced/added_tokens.json')

In [ ]:
# Save merged model for CPU
model_merged = model.merge_and_unload()
model_merged_fp32 = model_merged.to(torch.float32)
model_merged_fp32.save_pretrained("./biogpt-lora-balanced-merged")
tokenizer.save_pretrained("./biogpt-lora-balanced-merged")

('./biogpt-lora-balanced-merged/tokenizer_config.json',
 './biogpt-lora-balanced-merged/special_tokens_map.json',
 './biogpt-lora-balanced-merged/vocab.json',
 './biogpt-lora-balanced-merged/merges.txt',
 './biogpt-lora-balanced-merged/added_tokens.json')

### Comprehensive Metrics (20 samples)

In [ ]:
import evaluate
from tqdm import tqdm

In [ ]:
bleu_metric = evaluate.load("sacrebleu")
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
model_merged.eval()
sample_indices = np.random.choice(len(dataset['validation']), 20, replace=False)

predictions = []
references = []

for idx in tqdm(sample_indices, desc="Evaluating"):
    sample = dataset['validation'][int(idx)]

    instruction = sample.get('instruction', '')
    input_text = sample.get('input', '')
    output_text = sample.get('output', '')

    prompt = f"{instruction} {input_text}".strip() if input_text else instruction.strip()

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=256)
    inputs = {k: v.to(model_merged.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model_merged.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,  # Balanced creativity
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,  # Avoid repetition
            length_penalty=1.0,  # Natural length
            pad_token_id=tokenizer.pad_token_id
        )

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    prediction = generated[len(prompt):].strip() if generated.startswith(prompt) else generated.strip()

    predictions.append(prediction or "No response")
    references.append(output_text.strip())

# Calculate metrics
metrics = {}

try:
    bleu = bleu_metric.compute(predictions=predictions, references=[[r] for r in references])
    metrics['BLEU'] = bleu['score']
except:
    metrics['BLEU'] = 0.0

try:
    rouge = rouge_metric.compute(predictions=predictions, references=references)
    metrics['ROUGE-1'] = rouge['rouge1']
    metrics['ROUGE-2'] = rouge['rouge2']
    metrics['ROUGE-L'] = rouge['rougeL']
except:
    metrics['ROUGE-1'] = metrics['ROUGE-2'] = metrics['ROUGE-L'] = 0.0

try:
    meteor = meteor_metric.compute(predictions=predictions, references=references)
    metrics['METEOR'] = meteor['meteor']
except:
    metrics['METEOR'] = 0.0

Evaluating: 100%|██████████| 20/20 [00:34<00:00,  1.71s/it]


In [ ]:
for key, value in metrics.items():
    print(f"{key}: {value:.4f}")

BLEU: 2.0768
ROUGE-1: 0.2743
ROUGE-2: 0.0379
ROUGE-L: 0.1302
METEOR: 0.2364


### Quality Test with Analysis

In [ ]:
test_cases = [
    {
        "question": "What are the symptoms of diabetes?",
        "expected_keywords": ["thirst", "urination", "fatigue", "weight", "vision"],
    },
    {
        "question": "How to treat high blood pressure?",
        "expected_keywords": ["lifestyle", "diet", "exercise", "medication", "salt"],
    },
    {
        "question": "What causes migraine headaches?",
        "expected_keywords": ["trigger", "stress", "food", "hormonal", "sleep"],
    },
]

total_score = 0

for i, test in enumerate(test_cases, 1):
    prompt = test["question"]
    keywords = test["expected_keywords"]

    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(model_merged.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model_merged.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id
        )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = text[len(prompt):].strip().lower()

    # Check keyword coverage
    found_keywords = [kw for kw in keywords if kw in response]
    coverage = len(found_keywords) / len(keywords) * 100
    total_score += coverage

    print(f"\n{i}. Q: {prompt}")
    print(f"   A: {text[len(prompt):].strip()}")
    print(f"   Keyword coverage: {coverage:.0f}% ({len(found_keywords)}/{len(keywords)})")
    print(f"   Found: {', '.join(found_keywords)}")

avg_score = total_score / len(test_cases)
print(f"\n" + "=" * 70)
print(f"QUALITY SCORE: {avg_score:.1f}%")
print("=" * 70)
print("\n Expected: 60-80% coverage = Good quality")
print("   Higher coverage = More comprehensive responses")


1. Q: What are the symptoms of diabetes?
   A: If you are a doctor, please answer the medical questions based on the patient's description. Hello, I have been suffering from this problem for about 6 months now. I have had a routine test and blood sugar levels. The results show that they were not in normal range but there is some evidence to suggest a glucose level above which my blood sugar level goes up. They also feel very tired and exhausted when doing so. Please advise me what could be causing these problems. It would seem reasonable if I follow a diet low in fat and high in protein, as it has been shown that an increase in carbohydrate intake leads to more weight gain than with any other type of dietary change. You should also start taking supplements
   Keyword coverage: 20% (1/5)
   Found: weight

2. Q: How to treat high blood pressure?
   A: What should I do if you have been having headaches or any other symptoms. Hello, I am Chat Doctor. I am a female patient with low back pa

### Download

In [2]:
import shutil
from google.colab import files

In [ ]:
# For small download (LoRA adapter only - ~20MB):
shutil.make_archive('biogpt-lora', 'zip', './biogpt-lora-balanced')
files.download('biogpt-lora.zip')

In [ ]:
# For CPU inference (merged model - ~400MB):
shutil.make_archive('biogpt-merged', 'zip', './biogpt-lora-balanced-merged')
files.download('biogpt-merged.zip')